In [4]:
import requests
import json


# API_URL = "https://statsapi.web.nhl.com/api/v1/teams?expand=team.stats&season=19171918"

# Schedule for Dallas Stars 2014-15 season.
API_URL = "https://statsapi.web.nhl.com/api/v1/schedule?season=20142015&teamId=25"

response = requests.get(API_URL)

data = response.json()
# print(json.dumps(data, indent=1))

# print(json.dumps(data['teams'][1]['teamStats'][0]['splits'][0]['stat'], indent=1))

# Get the schedule for the 1917-18 season
# print(data['teams'][1]['teamStats'])

# print(data['teams'][1]['teamStats'][0])
# print(data['teams'][1]['teamStats'][0]['splits'])
# print(data['teams'][1]['teamStats'][0]['splits'][0])
# print(data['teams'][1]['teamStats'][0]['splits'][0]['stat'])
# print(data['teams'][1]['teamStats'][0]['splits'][0]['stat']['wins'])


nhl_teams_colors = {
    'Anaheim Ducks': '#f47a38',
    'Arizona Coyotes': '#8c2633',
    'Boston Bruins': '#fcb514',
    'Buffalo Sabres': '#002654',
    'Calgary Flames': '#c8102e',
    'Carolina Hurricanes': '#cc0000',
    'Chicago Blackhawks': '#cf0a2c',
    'Colorado Avalanche': '#6f263d',
    'Columbus Blue Jackets': '#00285c',
    'Dallas Stars': '#006847',
    'Detroit Red Wings': '#ce1126',
    'Edmonton Oilers': '#c8102e',
    'Florida Panthers': '#041e42',
    'Los Angeles Kings': '#a2aaad',
    'Minnesota Wild': '#154734',
    'Montréal Canadiens': '#af1e2d',
    'Nashville Predators': '#ffb81c',
    'New Jersey Devils': '#ce1126',
    'New York Islanders': '#f47d30',
    'New York Rangers': '#0038a8',
    'Ottawa Senators': '#c52032',
    'Philadelphia Flyers': '#f74902',
    'Pittsburgh Penguins': '#000000',
    'San Jose Sharks': '#006d75',
    'Seattle Kraken': '#82b3be',
    'St. Louis Blues': '#002f87',
    'Tampa Bay Lightning': '#002868',
    'Toronto Maple Leafs': '#003e7e',
    'Vancouver Canucks': '#001f5b',
    'Vegas Golden Knights': '#b4975a',
    'Washington Capitals': '#041e42',
    'Winnipeg Jets': '#041e42'
}



In [6]:
# Schedule for Dallas Stars 2014-15 season.
# API_URL = "https://statsapi.web.nhl.com/api/v1/schedule?season=20142015&teamId=25"

# Schedule for all teams 2014-15 season.
API_URL = "https://statsapi.web.nhl.com/api/v1/schedule?season=20222023"

response = requests.get(API_URL)

data = response.json()
# print(json.dumps(data, indent=1))
# print(type(data['dates'][0]))
# print(json.dumps(data['dates'][1], indent=1))

# data['dates'][0] gives you the first date.
# data['dates'][0]['games'] gives you the games for that date.

data['dates'][0]['games'] #[0]['teams']['away']['team']['name']

# Dictionary to hold team names and how many times they occur.
names = {}

# Iterate through all of the dates.
for i in range(len(data['dates'])):
    # Iterate through all of the games for that date.
    for j in range(len(data['dates'][i]['games'])):
        # If the game is a preseason, postseason, or all-star game, skip it.
        if data['dates'][i]['games'][j]['gameType'] == 'PR' or data['dates'][i]['games'][j]['gameType'] == 'P' or data['dates'][i]['games'][j]['gameType'] == 'A':
            continue

        # Get the away team name.
        away_team = data['dates'][i]['games'][j]['teams']['away']['team']['name']

        # Get the home team name.
        home_team = data['dates'][i]['games'][j]['teams']['home']['team']['name']

        # If there is no 'ot', print it and break.
        if 'ot' not in data['dates'][i]['games'][j]['teams']['away']['leagueRecord']:
            print(data['dates'][i]['games'][j]['teams']['away'])
            break

        # If there is no 'ot', print it and break.
        if 'ot' not in data['dates'][i]['games'][j]['teams']['home']['leagueRecord']:
            print(data['dates'][i]['games'][j]['teams']['home'])
            break

        # If the away team name is not in the dictionary, add it.
        if away_team not in names:
            names[away_team] = {'Games': 1}
            
            # Create new dictionary to keep track of the wins, losses, and ot losses.
            names[away_team]['Points'] = [sum([data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['wins'] * 2,\
                                              data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['losses'] * 0,\
                                              data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['ot'] * 1])]
        # If the away team name is in the dictionary, increment the count.
        else:
            names[away_team]['Games'] += 1
            names[away_team]['Points'].append(sum([data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['wins'] * 2,\
                                                data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['losses'] * 0,\
                                                data['dates'][i]['games'][j]['teams']['away']['leagueRecord']['ot'] * 1]))

        # If the home team name is not in the dictionary, add it.
        if home_team not in names:
            names[home_team] = {'Games': 1}
            names[home_team]['Points'] = [sum([data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['wins'] * 2,\
                                    data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['losses'] * 0,\
                                    data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['ot'] * 1])]
        # If the home team name is in the dictionary, increment the count.
        else:
            names[home_team]['Games'] += 1
            names[home_team]['Points'].append(sum([data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['wins'] * 2,\
                                                  data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['losses'] * 0,\
                                                  data['dates'][i]['games'][j]['teams']['home']['leagueRecord']['ot'] * 1]))

        # Get the away team score.
        away_score = data['dates'][i]['games'][j]['teams']['away']['score']
        # Get the home team score.
        home_score = data['dates'][i]['games'][j]['teams']['home']['score']
        # Print the game information.
        # print(away_team, away_score, home_team, home_score)

In [11]:
# Now that I have the data, plot it.
import plotly.graph_objs as go
import plotly.offline as pyo
import numpy as np

# Max games played.
max_games = 0
for team_name in names.keys():
    if names[team_name]['Games'] > max_games:
        max_games = names[team_name]['Games']

# Define your x and y values for each line
x_values = np.arange(1, max_games)
data = []

# Define the traces for each line
for team_name in names.keys():
    data.append(go.Scatter(x=x_values, y=names[team_name]['Points'], name=team_name, mode='lines', line_color=nhl_teams_colors[team_name]))
# trace_1 = go.Scatter(x=x_values, y=names['Anaheim Ducks']['Points'], name='Anaheim Ducks')
# trace_2 = go.Scatter(x=x_values, y=names['Dallas Stars']['Points'], name='Dallas Stars')
# trace_3 = go.Scatter(x=x_values, y=names['Los Angeles Kings']['Points'], name='Los Angeles Kings')

# Define the layout of the plot
layout = go.Layout(title='Team Point Progression', xaxis=dict(title='Games Played'), yaxis=dict(title='Number of Points'))

# Define the data to be plotted
# data = [trace_1, trace_2, trace_3]

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Display the plot
pyo.plot(fig, filename='multiple-line-graph.html')


'multiple-line-graph.html'

In [28]:
names.keys()

dict_keys(['Montréal Canadiens', 'Toronto Maple Leafs', 'Philadelphia Flyers', 'Boston Bruins', 'Vancouver Canucks', 'Calgary Flames', 'San Jose Sharks', 'Los Angeles Kings', 'Columbus Blue Jackets', 'Buffalo Sabres', 'New Jersey Devils', 'Anaheim Ducks', 'Pittsburgh Penguins', 'Washington Capitals', 'Detroit Red Wings', 'Florida Panthers', 'Tampa Bay Lightning', 'New York Rangers', 'St. Louis Blues', 'Ottawa Senators', 'Nashville Predators', 'Chicago Blackhawks', 'Dallas Stars', 'Colorado Avalanche', 'Minnesota Wild', 'Edmonton Oilers', 'Winnipeg Jets', 'Arizona Coyotes', 'New York Islanders', 'Carolina Hurricanes'])